In [1]:
import numpy as np
from glob import glob
import torch
import os, sys, time, csv
import re
import random
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as ipyw
import inspect
import pickle
%matplotlib inline

# Process image stacks into feature vector matrices using pre-trained model (for LSTM input).

In [4]:
# Torch imports.
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision.models.resnet

# Visualization imports
from IPython import display


In [5]:
# Part A: Load in model pre-trained on individual images. Note that pre-training may not even be necessary, as I was only
# changing the weights on the last (new) fully connected layer, and that layer will likely be gotten rid of. 
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

# Assemble image pathnames and understand class breakdown in dataset

In [6]:
# Note down train, validation and test directories.
topDir = '/home/ubuntu/src/YPB-AI/Nate/Scripts/Bizcards_split' # without PanPass and pillBottle
# topDir = 'Split_Data2_23_11_19'# with PanPass
# topDir = 'Split_Data_bamData'

train_dir = topDir + "/train"
valid_dir = topDir + "/valid"
test_dir = topDir + "/test"
# test_dir = 'Bam1-CIFAR'

# Check quantities of train, validation and test images
train_images = np.array(glob(train_dir + "/*/*"))
valid_images = np.array(glob(valid_dir + "/*/*"))
test_images = np.array(glob(test_dir + "/*/*"))

# Check relative percentages of image types
train_images_absent = np.array(glob(train_dir + "/Absent/*"))
train_images_present = np.array(glob(train_dir + "/Present/*"))

valid_images_absent = np.array(glob(valid_dir + "/Absent/*"))
valid_images_present = np.array(glob(valid_dir + "/Present/*"))

test_images_absent = np.array(glob(test_dir + "/Absent/*"))
test_images_present = np.array(glob(test_dir + "/Present/*"))

num_train_images = len(train_images)
num_valid_images = len(valid_images)
num_test_images = len(test_images)

print("There are {} training stacks, {} validation stacks and {} test stacks.".format(len(train_images),len(valid_images),len(test_images)))
print("For the training stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(train_images_present)/len(train_images)*100, pos2=len(train_images_present),neg=len(train_images_absent)/len(train_images)*100, neg2=len(train_images_absent)))
print("For the validation stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(valid_images_present)/len(valid_images)*100, pos2=len(valid_images_present),neg=len(valid_images_absent)/len(valid_images)*100, neg2=len(valid_images_absent)))
print("For the test stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(test_images_present)/len(test_images)*100, pos2=len(test_images_present),neg=len(test_images_absent)/len(test_images)*100, neg2=len(test_images_absent)))

There are 4058 training stacks, 507 validation stacks and 507 test stacks.
For the training stacks, 53.6% (2174) are positive and 46.4% (1884) are negative.
For the validation stacks, 53.6% (272) are positive and 46.4% (235) are negative.
For the test stacks, 53.6% (272) are positive and 46.4% (235) are negative.


In [7]:
# Check for GPU support
use_cuda = torch.cuda.is_available()

In [8]:
# Check to see how many GPUs are available.
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    num_devices = torch.cuda.device_count()
    print("{} GPUs found.".format(num_devices))
else:
    num_devices = 0

1 GPUs found.


# Imports for torch and DALI functions

In [9]:
import torch.nn.functional as F

# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(224, 224, 3)
#         self.conv2 = nn.Conv2d(224, 224, 64) #2 layers
#         # http://web.media.mit.edu/~pratiks/combined-classification/convolutional-neural-network-for-combined-classification-of-fluorescent-biomarkers-and-expert-annotations-using-white-light-images.pdf
#         # https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# net = Net()

In [10]:
# Define the model
class MM_DNN(nn.Module):
    def __init__(self, drop_prob=0.2, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.lr = lr
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(4*1024,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,2)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        x = x.view(-1)
        out = self.dropout(self.relu(self.fc(x)))
        out = self.dropout(self.relu(self.fc2(out)))
        out = self.fc3(out)
        # return the final output
        return out
    


In [11]:
# Use the PyTorch data loader here, way easier. 
def load_pickle(path):
    with open(path,'rb') as f:
        return pickle.load(f)

In [12]:
train_data = datasets.DatasetFolder(train_dir,load_pickle,'.pickle')
validation_data = datasets.DatasetFolder(valid_dir,load_pickle,'.pickle')
test_data = datasets.DatasetFolder(test_dir,load_pickle,'.pickle')
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 1, shuffle=True, drop_last=True, num_workers=num_devices*4, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(validation_data, batch_size = 1, num_workers=num_devices*4, pin_memory=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 1, num_workers=num_devices*4, pin_memory=True)

In [13]:
x = load_pickle('/home/ubuntu/src/YPB-AI/Nate/Bizcards/Absent/Absent_19-Feb-2020_09-02-16.pickle')
x.shape
# image, label = iter(train_loader).next()
# x

torch.Size([0])

In [14]:
def imshow(img):
    # img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()[0]
    print(npimg.shape)
    plt.imshow(np.transpose(npimg,(1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
print(labels)
# print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

(0,)


ValueError: axes don't match array

In [13]:
saveName = 'Testing_CIFAR_all_18_01_20'

In [14]:
import torch.nn as nn
import torch.nn.functional as F

# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html 
class CIFARNet1(nn.Module):
    def __init__(self):
        super(CIFARNet1, self).__init__()
        self.conv1 = nn.Conv2d(12, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = CIFARNet1()

In [37]:
# define and print the net

# net = MM_DNN()
net = CIFARNet1()
print(net)

CIFARNet1(
  (conv1): Conv2d(12, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [16]:
train_loader.dataset.samples[batch_idx][0]
y = load_pickle('/home/ubuntu/src/YPB-AI/Split_Data_bamData/train/Absent/6S_Negative_Dark_14-Mar-2019_11-21-40.pickle')
y.shape
batch_data

NameError: name 'batch_idx' is not defined

In [33]:
# Load current weights if exist
home_folder = '/home/ubuntu/src/YPB-AI/'
init_saveName = 'Testing_bamData'
net = torch.load(home_folder + 'DNNModel_' + init_saveName + '.pt')
init_stateDictName = home_folder + 'DNN_StateDict_' + init_saveName + '.pt'
net.load_state_dict(torch.load(init_stateDictName))

In [38]:
epochs=50
batch_size=1
lr=0.0001
print_every=6000
valid_batch = 1
net.train()

opt = torch.optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

if(use_cuda):
    net.cuda()
all_valid_losses = [.6]
counter = 0
for e in range(epochs):

    for batch_idx, batch_data in enumerate(train_loader):
        # print(batch_data[0][0].shape)
        # inputs = batch_data[0][0].view(12,batch_size,3,32,32)
        inputs = batch_data[0][0]
        inputs = inputs[0:4]
        # print(inputs.shape)
        try:
            inputs = inputs.view(batch_size,12,32,32)
        except:
            print(train_loader.dataset.samples[batch_idx][0])
            continue
        targets = batch_data[1]
        if(use_cuda):
            inputs, targets = inputs.cuda(), targets.cuda()
        # zero accumulated gradients
        net.zero_grad()
        # get the output from the model
        output = net(inputs)
        # calculate the loss and perform backprop
        loss = criterion(output.view(batch_size,2), targets)
        loss.backward()
        opt.step()
        # loss stats
        if batch_idx % print_every == 0:
            # Get validation loss
            val_losses = []
            net.eval()
            correct = 0.
            total = 0.
            for val_batch_idx, batch_data in enumerate(valid_loader): 
                # inputs, targets = batch_data[0].view(12,valid_batch,1024), batch_data[1]
                inputs, targets = batch_data[0][0], batch_data[1]
                inputs = inputs[0:4]
                try:
                    inputs = inputs.view(batch_size,12,32,32)
                except:
                    print(train_loader.dataset.samples[batch_idx][0])
                    continue

                if(use_cuda):
                    inputs, targets = inputs.cuda(), targets.cuda()

                output = net(inputs)
                # print(output)
                
                val_loss = criterion(output.view(valid_batch,2), targets)
                val_losses.append(val_loss.item())
                pred = output[0].max(0,keepdim=True)[1]
                # print(pred)
                # sm = nn.Softmax(dim=1)
                # sm_outputs = sm(output)
                # print(sm_outputs)
                # compare predictions to true label
                correct += np.sum(np.squeeze(pred.eq(targets.data.view_as(pred))).cpu().numpy())
                total += valid_batch

            net.train() # reset to train mode after iteration through validation data

            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(batch_idx),
                  "Loss: {:.4f}...".format(loss.item()),
                  "Val Loss: {:.4f}...".format(np.mean(val_losses)),
                  "Val Accuracy: {:.1f}%".format(correct/total*100))

            if np.mean(val_losses) < np.min(all_valid_losses):
                print("Validation loss dropped, saving model.")
                torch.save(net,'DNNModel_' + saveName + '.pt')
                # https://stackoverflow.com/questions/52277083/pytorch-saving-model-userwarning-couldnt-retrieve-source-code-for-container-of
                all_valid_losses = np.append(all_valid_losses,np.mean(val_losses))

Epoch: 1/50... Step: 0... Loss: 0.7884... Val Loss: 0.6993... Val Accuracy: 49.2%
Epoch: 1/50... Step: 6000... Loss: 0.7413... Val Loss: 0.6118... Val Accuracy: 65.0%
Epoch: 1/50... Step: 12000... Loss: 0.4082... Val Loss: 0.5877... Val Accuracy: 68.0%
Validation loss dropped, saving model.


/home/ubuntu/anaconda3/envs/derm-ai/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type CIFARNet1. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1/50... Step: 18000... Loss: 0.5000... Val Loss: 0.5555... Val Accuracy: 70.8%
Validation loss dropped, saving model.
Epoch: 2/50... Step: 0... Loss: 0.6706... Val Loss: 0.5507... Val Accuracy: 66.1%
Validation loss dropped, saving model.
Epoch: 2/50... Step: 6000... Loss: 0.1133... Val Loss: 0.5357... Val Accuracy: 70.8%
Validation loss dropped, saving model.
Epoch: 2/50... Step: 12000... Loss: 0.4216... Val Loss: 0.5322... Val Accuracy: 70.7%
Validation loss dropped, saving model.
Epoch: 2/50... Step: 18000... Loss: 0.8693... Val Loss: 0.5053... Val Accuracy: 74.9%
Validation loss dropped, saving model.
Epoch: 3/50... Step: 0... Loss: 0.5731... Val Loss: 0.4963... Val Accuracy: 74.9%
Validation loss dropped, saving model.
Epoch: 3/50... Step: 6000... Loss: 0.6213... Val Loss: 0.4579... Val Accuracy: 78.2%
Validation loss dropped, saving model.
Epoch: 3/50... Step: 12000... Loss: 0.2351... Val Loss: 0.4931... Val Accuracy: 75.4%
Epoch: 3/50... Step: 18000... Loss: 0.0032... Val

Epoch: 22/50... Step: 12000... Loss: 0.4274... Val Loss: 0.2299... Val Accuracy: 88.8%
Epoch: 22/50... Step: 18000... Loss: 0.0000... Val Loss: 0.2160... Val Accuracy: 91.2%
Epoch: 23/50... Step: 0... Loss: 0.0014... Val Loss: 0.2125... Val Accuracy: 91.6%
Epoch: 23/50... Step: 6000... Loss: 0.0008... Val Loss: 0.2130... Val Accuracy: 91.0%
Epoch: 23/50... Step: 12000... Loss: 0.7548... Val Loss: 0.2156... Val Accuracy: 89.8%
Epoch: 23/50... Step: 18000... Loss: 0.0003... Val Loss: 0.2277... Val Accuracy: 88.6%
Epoch: 24/50... Step: 0... Loss: 0.5500... Val Loss: 0.2062... Val Accuracy: 89.5%
Epoch: 24/50... Step: 6000... Loss: 0.8310... Val Loss: 0.2236... Val Accuracy: 90.0%
Epoch: 24/50... Step: 12000... Loss: 0.1855... Val Loss: 0.2057... Val Accuracy: 90.9%
Epoch: 24/50... Step: 18000... Loss: 0.0000... Val Loss: 0.1987... Val Accuracy: 91.2%
Validation loss dropped, saving model.
Epoch: 25/50... Step: 0... Loss: 0.0000... Val Loss: 0.2187... Val Accuracy: 89.6%
Epoch: 25/50... St

Epoch: 46/50... Step: 0... Loss: 0.0242... Val Loss: 0.2978... Val Accuracy: 87.5%
Epoch: 46/50... Step: 6000... Loss: 0.0000... Val Loss: 0.2760... Val Accuracy: 90.5%
Epoch: 46/50... Step: 12000... Loss: 0.0000... Val Loss: 0.2126... Val Accuracy: 90.6%
Epoch: 46/50... Step: 18000... Loss: 0.5327... Val Loss: 0.1916... Val Accuracy: 92.2%
Epoch: 47/50... Step: 0... Loss: 0.0000... Val Loss: 0.2357... Val Accuracy: 92.2%
Epoch: 47/50... Step: 6000... Loss: 0.0000... Val Loss: 0.2553... Val Accuracy: 91.2%
Epoch: 47/50... Step: 12000... Loss: 0.3601... Val Loss: 0.3145... Val Accuracy: 89.2%
Epoch: 47/50... Step: 18000... Loss: 0.0135... Val Loss: 0.2361... Val Accuracy: 93.0%
Epoch: 48/50... Step: 0... Loss: 0.0000... Val Loss: 0.2240... Val Accuracy: 90.2%
Epoch: 48/50... Step: 6000... Loss: 0.0000... Val Loss: 0.2153... Val Accuracy: 92.4%
Epoch: 48/50... Step: 12000... Loss: 0.0005... Val Loss: 0.2080... Val Accuracy: 92.8%
Epoch: 48/50... Step: 18000... Loss: 0.0000... Val Loss: 0

In [19]:
stateDictName ='DNN_StateDict_' + saveName + '.pt'
torch.save(net.state_dict(), stateDictName)

## Test on test data

In [117]:
saveName = 'Testing_bamData'
# saveName = 'Testing_CIFAR_all_18_01_20'
# saveName = 'Testing_CIFAR_iter_18_01_20'

In [47]:
net = CIFARNet1()
print(net)

CIFARNet1(
  (conv1): Conv2d(12, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [119]:
# net = torch.load('DNNModel_' + saveName + '.pt')
stateDictName ='DNN_StateDict_' + saveName + '.pt'

In [42]:
torch.save(net.state_dict(), stateDictName)

In [120]:
net.load_state_dict(torch.load('/home/ubuntu/src/YPB-AI/' + stateDictName))

In [16]:
# Test with all data
test_dir = '/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/Training/test'
test_data = datasets.DatasetFolder(test_dir,load_pickle,'.pickle')
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 1, shuffle=True, drop_last=True, num_workers=num_devices*4, pin_memory=True)

In [22]:
test_loader.dataset.samples[test_batch][0]
print(inputs.shape)
output.shape

NameError: name 'test_batch' is not defined

In [121]:
# Test CIFAR
net.eval()
correct = 0.
total = 0.
false_pos = 0.
false_neg = 0.
test_batch = 1
test_losses = []
batch_size = 1
criterion = nn.CrossEntropyLoss()

false_pos_img = []
false_neg_img = []

for test_batch_idx, batch_data in enumerate(test_loader):                    
    inputs, targets = batch_data[0][0], batch_data[1]
    inputs = inputs[0:4]
    inputs = inputs.view(batch_size,12,32,32)

    if(use_cuda):
        inputs, targets = inputs.cuda(), targets.cuda()
        net.cuda()

    # print(inputs)
    output = net(inputs)

    test_loss = criterion(output.view(test_batch,2), targets)
    test_losses.append(test_loss.item()/len(test_loader))

    pred = output[0].max(0,keepdim=True)[1]
    
    # compare predictions to true label
    correct += np.sum(np.squeeze(pred.eq(targets.data.view_as(pred))).cpu().numpy())
    
    # number of false positives; hack for batch size = 1
    # false_pos += int(targets.data.cpu().numpy()[0] == 0 and pred.data.cpu().numpy()[0] == 1)
    if targets.data.cpu().numpy()[0] == 0 and pred.data.cpu().numpy()[0] == 1:
        false_pos += 1
        false_pos_img.append(test_loader.dataset.samples[test_batch_idx][0])
    
    # number of false negatives; hack for batch size = 1
    # false_neg += int(targets.data.cpu().numpy()[0] == 1 and pred.data.cpu().numpy()[0] == 0)
    if targets.data.cpu().numpy()[0] == 1 and pred.data.cpu().numpy()[0] == 0:
        false_neg += 1
        false_neg_img.append(test_loader.dataset.samples[test_batch_idx][0])
    
    total += test_batch

In [122]:
print(correct)
print(total)
print(false_pos)
print(false_neg)
int(targets.data.cpu().numpy()[0] == 1 and pred.data.cpu().numpy()[0] == 0)

2036.0
2650.0
43.0
571.0


1

In [28]:
# View images that are false negatives
# print(false_neg_img)

top_folder = 'Training_images/'
basewidth = 100

for pickle_file in false_neg_img:
    print(pickle_file)
    if '30-Oct-2019' in pickle_file:
        top_folder = '30_10_62_original/'
    elif '29-Oct-2019' in pickle_file:
        top_folder = '29_10_62_original/'
    elif '06-Nov-2019' in pickle_file:
        top_folder = 'Data/6_11_62/'
    elif '05-Nov-2019' in pickle_file:
        top_folder = 'Data/5_11_62/'
    else: 
        top_folder = 'Training_images/'
    folder_and_file_name = pickle_file.split('/')[3]
    delimiter_bits = re.split("\w*_", folder_and_file_name) # my failed attempt to properly use regular expressions
    folder = folder_and_file_name.split(delimiter_bits[1])[0][:-1]
    datetime = delimiter_bits[1] + '2019_' + delimiter_bits[2].split('.pickle')[0]
    img_folder = top_folder + folder + '/ProtectCodeScannerResults_' + datetime + '/Photos'
    print(img_folder)
    try: 
        idx = 1
        plt.figure(figsize=(20,40))
        for img_file in os.listdir(img_folder):
            if img_file.endswith(".JPG"): 
                # print(img_folder + '/' + img_file)
                img = Image.open(img_folder + '/' + img_file)
                # wpercent = (basewidth/float(img.size[0]))
                # hsize = int((float(img.size[1])*float(wpercent)))
                # img = img.resize((basewidth,hsize), Image.ANTIALIAS)
                plt.subplot(1, 12, idx)
                plt.imshow(img)
                idx += 1
        plt.show()
    except:
        # print('error')
        continue

In [30]:
# View false positives
# print(false_pos_img)

# basewidth = 100

for pickle_file in false_pos_img:
    print(pickle_file)
#     if '30-Oct-2019' in pickle_file:
#         top_folder = '30_10_62_original/'
#     elif '29-Oct-2019' in pickle_file:
#         top_folder = '29_10_62_original/'
#     else: 
#         top_folder = 'Training_images/'
#     folder_and_file_name = pickle_file.split('/')[3]
#     delimiter_bits = re.split("\w*_", folder_and_file_name) # my failed attempt to properly use regular expressions
#     folder = folder_and_file_name.split(delimiter_bits[1])[0][:-1]
#     datetime = delimiter_bits[1] + '2019_' + delimiter_bits[2].split('.pickle')[0]
#     img_folder = top_folder + folder + '/ProtectCodeScannerResults_' + datetime + '/Photos'
#     print(img_folder)
#     try: 
#         idx = 1
#         plt.figure(figsize=(20,40))
#         for img_file in os.listdir(img_folder):
#             if img_file.endswith(".JPG"): 
#                 # print(img_folder + '/' + img_file)
#                 img = Image.open(img_folder + '/' + img_file)
#                 # wpercent = (basewidth/float(img.size[0]))
#                 # hsize = int((float(img.size[1])*float(wpercent)))
#                 # img = img.resize((basewidth,hsize), Image.ANTIALIAS)
#                 plt.subplot(1, 12, idx)
#                 plt.imshow(img)
#                 idx += 1
#         plt.show()
#     except:
#         print('error')
#         continue

/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-24-38.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-24-55.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-25-37.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-35-37.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-45-53.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-50-33.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-51-31.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-52-04.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CIFAR/all_data/test/Absent/6S_Negative_Dark_14-Mar-2019_09-54-49.pickle
/home/ubuntu/src/YPB-AI/afterSplit/CI

## Test MM_DNN (Baseline)

In [48]:
saveName = 'Testing'

In [49]:
net = MM_DNN()
print(net)

MM_DNN(
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=4096, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)


In [50]:
net = torch.load('DNNModel_' + saveName + '.pt')
stateDictName ='DNN_StateDict_' + saveName + '.pt'

In [52]:
# Test with all data
test_dir = 'Training'
test_data = datasets.DatasetFolder(test_dir,load_pickle,'.pickle')
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 1, shuffle=True, drop_last=True, num_workers=num_devices*4, pin_memory=True)

In [53]:
net.eval()
correct = 0.
total = 0.
false_pos = 0.
false_neg = 0.
test_batch = 1
test_losses = []
criterion = nn.CrossEntropyLoss()
for test_batch_idx, batch_data in enumerate(test_loader):                    
    inputs, targets = batch_data[0].view(12,test_batch,1024), batch_data[1]
    inputs = inputs[0:4]

    if(use_cuda):
        inputs, targets = inputs.cuda(), targets.cuda()

    # print(inputs)
    output = net(inputs)

    test_loss = criterion(output.view(test_batch,2), targets)
    test_losses.append(test_loss.item()/len(test_loader))

    pred = output.max(0,keepdim=True)[1]
    
    # compare predictions to true label
    correct += np.sum(np.squeeze(pred.eq(targets.data.view_as(pred))).cpu().numpy())
    
    # number of false positives; hack for batch size = 1
    false_pos += int(targets.data.cpu().numpy()[0] == 0 and pred.data.cpu().numpy()[0] == 1)
    
    # number of false negatives; hack for batch size = 1
    false_neg += int(targets.data.cpu().numpy()[0] == 1 and pred.data.cpu().numpy()[0] == 0)
    
    total += test_batch

In [47]:
# MM_DNN
print(correct)
print(total)
print(false_pos)
print(false_neg)
int(targets.data.cpu().numpy()[0] == 1 and pred.data.cpu().numpy()[0] == 0)

11269.0
11405.0
88.0
48.0


0

In [37]:
net = torch.load('DNNModel_' + saveName + '.pt')
stateDictName ='DNN_StateDict_' + saveName + '.pt'

In [38]:
torch.save(net.state_dict(), stateDictName)

In [39]:
net.load_state_dict(torch.load(stateDictName))

In [40]:
net.load_state_dict(torch.load('DNN_StateDict_S3_Iter_1.pt'))

# Zip up state dict, get over to S3

In [90]:
# The below lines are only needed if allowing SageMaker to perform inference on the model.
# I can walk you through that when you get there.

#tarName = 'DNNModel_' + saveName + '.tar.gz' 

In [91]:
#os.system('tar -cvzf ' + tarName + ' ' + stateDictName)
#os.system('aws s3 cp ' + tarName + ' s3://ypb-ml-images/' + tarName)

0